In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.linalg import expm, eigh  # Use scipy for the matrix exponential function

def qr_algorithm(A, tol=1e-10, max_iter=1000):
    """
    Computes the eigenvalues of a square matrix A using the QR algorithm.
    
    Parameters:
        A (numpy.ndarray): The input square matrix (n x n).
        tol (float): Convergence tolerance for off-diagonal elements.
        max_iter (int): Maximum number of iterations.
    
    Returns:
        numpy.ndarray: Array of eigenvalues.
    """
    # Ensure the matrix is square
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix A must be square")
    
    # Copy A to avoid modifying the original matrix
    A_k = A.copy()
    
    for k in range(max_iter):
        # Shift: Use the bottom-right element of A_k as the shift
        mu = A_k[-1, -1]
        # Perform QR decomposition of (A_k - mu * I)
        Q, R = np.linalg.qr(A_k - mu * np.eye(n))
        # Update A_k
        A_k = R @ Q + mu * np.eye(n)
        
        # Check for convergence
        off_diagonal_norm = np.sqrt(np.sum(np.tril(A_k, -1)**2))
        if off_diagonal_norm < tol:
            break
    else:
        print("QR algorithm did not converge within the maximum number of iterations.")
    
    # Eigenvalues are the diagonal elements of A_k
    return np.diag(A_k)


In [8]:
# Example usage:
A = np.array([[6, 2, 1],
              [2, 3, 1],
              [1, 1, 1]], dtype=float)

eigenvalues = qr_algorithm(A)
print("Eigenvalues:", eigenvalues)

Eigenvalues: [7.28799214 2.13307448 0.57893339]


In [6]:
class Simulator:
    def __init__(self, N, H):
        self.N = N
        self.H = H
    
    def compute_eigenvalues(self):
        """Compute all possible eigenvalues of the Hamiltonian."""
        eigenvalues = set()
        for config in range(1 << self.N):
            spins = [1 if (config & (1 << i)) else -1 for i in range(self.N)]
            eigenvalue = self.H(spins)
            eigenvalues.add(eigenvalue)
        return sorted(eigenvalues)

In [7]:
class NumberPartitioning:
    def __init__(self, numbers):
        self.numbers = numbers
    
    def hamiltonian(self):
        """Define the Ising Hamiltonian for the problem."""
        # H = A * (sum(n_i * s_i))^2
        return lambda spins: (np.sum(np.array(self.numbers) * np.array(spins))) ** 2
    
numbers = [3, 1, 4, 2, 2]
np_problem = NumberPartitioning(numbers)
sim = Simulator(len(numbers), np_problem.hamiltonian())

eiganvalues = sim.compute_eigenvalues()
print(eiganvalues)

[0, 4, 16, 36, 64, 100, 144]


In [17]:
num_states = 2**sim.N    
H_final = np.zeros((num_states, num_states), dtype=complex)

# Construct the target (final) Hamiltonian matrix
for config in range(num_states):
    spins = [1 if (config & (1 << i)) else -1 for i in range(sim.N)]
    energy = sim.H(spins)
    H_final[config, config] = energy
print(H_final)
print(f"Number of spins: {sim.N}, Matrix size: {2**sim.N} x {2**sim.N}")

[[144.+0.j   0.+0.j   0.+0.j ...   0.+0.j   0.+0.j   0.+0.j]
 [  0.+0.j  36.+0.j   0.+0.j ...   0.+0.j   0.+0.j   0.+0.j]
 [  0.+0.j   0.+0.j 100.+0.j ...   0.+0.j   0.+0.j   0.+0.j]
 ...
 [  0.+0.j   0.+0.j   0.+0.j ... 100.+0.j   0.+0.j   0.+0.j]
 [  0.+0.j   0.+0.j   0.+0.j ...   0.+0.j  36.+0.j   0.+0.j]
 [  0.+0.j   0.+0.j   0.+0.j ...   0.+0.j   0.+0.j 144.+0.j]]
Number of spins: 5, Matrix size: 32 x 32


In [18]:
eigenvalues = qr_algorithm(H_final)
print("Eigenvalues:", eigenvalues)

Eigenvalues: [144.+0.j  36.+0.j 100.+0.j  16.+0.j  16.+0.j   4.+0.j   4.+0.j  16.+0.j
  64.+0.j   4.+0.j  36.+0.j   0.+0.j   0.+0.j  36.+0.j   4.+0.j  64.+0.j
  64.+0.j   4.+0.j  36.+0.j   0.+0.j   0.+0.j  36.+0.j   4.+0.j  64.+0.j
  16.+0.j   4.+0.j   4.+0.j  16.+0.j  16.+0.j 100.+0.j  36.+0.j 144.+0.j]


In [21]:
import numpy as np

def lanczos_eigenvalues(A, k=None, tol=1e-10):
    """
    Computes the largest k eigenvalues of a symmetric/Hermitian matrix A using the Lanczos algorithm.
    Handles both real and complex matrices.
    
    Parameters:
        A (numpy.ndarray): The input symmetric (real) or Hermitian (complex) matrix (n x n).
        k (int): Number of largest eigenvalues to compute. Defaults to n.
        tol (float): Convergence tolerance.
    
    Returns:
        numpy.ndarray: Array of the largest k eigenvalues.
    """
    n = A.shape[0]
    
    # Ensure matrix is square
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix A must be square")
    
    # If k is not specified, compute all eigenvalues
    if k is None or k > n:
        k = n

    # Initial vector (randomized for robustness), allowing complex numbers
    v = np.random.rand(n) + 1j * np.random.rand(n) if np.iscomplexobj(A) else np.random.rand(n)
    v /= np.linalg.norm(v)

    # Initialize variables
    T = np.zeros((k, k), dtype=A.dtype)  # Tridiagonal matrix
    V = np.zeros((n, k), dtype=A.dtype)  # Lanczos vectors
    beta = 0
    q_prev = np.zeros(n, dtype=A.dtype)
    
    for j in range(k):
        # Compute the Lanczos vector
        w = A @ v - beta * q_prev
        alpha = np.vdot(v, w)  # Use Hermitian inner product for complex matrices
        w -= alpha * v
        beta = np.linalg.norm(w)
        
        # Reorthogonalize if necessary
        if beta < tol:
            break
        
        # Update matrices
        T[j, j] = alpha
        if j < k - 1:
            T[j, j + 1] = beta
            T[j + 1, j] = beta
        V[:, j] = v
        q_prev = v
        v = w / beta

    # Solve the tridiagonal eigenproblem
    eigvals, _ = np.linalg.eigh(T[:j+1, :j+1])
    
    return eigvals


In [22]:
eigenvalues = lanczos_eigenvalues(H_final)
print("Eigenvalues:", eigenvalues)

Eigenvalues: [ -4.22987996   1.64054703  14.41929779  35.68710798  63.97012304
  99.98527072 143.99993012]


In [24]:
# Use the Lanczos algorithm with tightened parameters
eigenvalues = lanczos_eigenvalues(H_final, k=H_final.shape[0], tol=1e-12)

# Compare with exact eigenvalues
exact_eigenvalues = np.linalg.eigh(H_final)[0]
print("Lanczos Eigenvalues:", eigenvalues)
print("Exact Eigenvalues:", exact_eigenvalues)

Lanczos Eigenvalues: [-3.32729023e-14 -2.77759930e-14 -1.23575463e-14  1.51608071e-14
  4.00000000e+00  4.00000000e+00  4.00000000e+00  4.00000000e+00
  1.60000000e+01  1.60000000e+01  1.60000000e+01  1.60000000e+01
  3.59999881e+01  3.60000000e+01  3.60000000e+01  3.60000000e+01
  3.60000000e+01  6.39999842e+01  6.40000000e+01  6.40000000e+01
  6.40000000e+01  6.40000000e+01  1.00000000e+02  1.00000000e+02
  1.00000000e+02  1.00000000e+02  1.00000000e+02  1.44000000e+02
  1.44000000e+02  1.44000000e+02  1.44000000e+02  1.44000000e+02]
Exact Eigenvalues: [  0.   0.   0.   0.   4.   4.   4.   4.   4.   4.   4.   4.  16.  16.
  16.  16.  16.  16.  36.  36.  36.  36.  36.  36.  64.  64.  64.  64.
 100. 100. 144. 144.]
